In [2]:
import jieba
jieba.set_dictionary('ChineseDictionary/dict.txt.big')
import numpy as np
import pandas as pd
import operator
import matplotlib.pyplot as plt
%matplotlib inline
data = pd.read_excel("customer_record_data.xlsx")

In [4]:
data.head()

,類別,Unique ID,資料日期,客戶事件描述
0,Call Out,AA2765,20180102 141000,[ECP] 去電確認開戶文件 客戶說後來去買美元保單就不開戶了
1,Call Out,AA3910,20180102 144300,[ECP]不方說 IPO MAIL
2,Call Out,AA3167,20180102 154500,[ECP] IPO 忙 先MAIL
3,Call Out,AA2355,20180102 194359,家庭戶 媽媽說女兒不開了，因為打工收入不穩定
4,Call Out,AA2778,20180103 111132,推Q1


First, lets see how many unique IDs are there in the record

In [5]:
data = np.array(data)
unique_ids = set(data[:,1])
print("Amount of Unique IDs: "+str(len(unique_ids)))

Amount of Unique IDs: 572


Then we count times of the ID (we will call it customer) had called.

In [6]:
unique_id_called = dict()
for i in data:
    if i[1] in unique_id_called:
        unique_id_called[i[1]] += 1
    else:
        unique_id_called[i[1]] = 1

sorted_unique_id_called = sorted(unique_id_called.items(),key=operator.itemgetter(1))
# for i in range(1,len(unique_id_called)):
#     print(sorted_unique_id_called[-i])

draw_hist = dict()
for i in sorted_unique_id_called:
    if i[1] in draw_hist:
        draw_hist[i[1]] += 1
    else:
        draw_hist[i[1]] = 1
        
keys = np.array(list(draw_hist.keys()))
values = np.array(list(draw_hist.values( ) ) )
for i in range(len(keys)):
    print("called",keys[i],"time(s) for",values[i],"times")


called 1 time(s) for 350 times
called 2 time(s) for 123 times
called 3 time(s) for 50 times
called 4 time(s) for 20 times
called 5 time(s) for 15 times
called 6 time(s) for 8 times
called 7 time(s) for 4 times
called 8 time(s) for 1 times
called 14 time(s) for 1 times


Now, we look at some of the most common word in the message section.

In [24]:
import jieba.analyse
jieba.enable_parallel(8)
cnt = 0
for i in data:
    cnt = cnt + 1
    if cnt <= 15:
        print(jieba.lcut(i[3]))

['[', 'ECP', ']', ' ', '去電', '確認', '開戶', '文件', ' ', '客戶', '說', '後來', '去', '買', '美元', '保單', '就', '不', '開戶', '了']
['[', 'ECP', ']', '不方', '說', ' ', ' ', 'IPO', ' ', 'MAIL']
['[', 'ECP', ']', ' ', 'IPO', ' ', '忙', ' ', '先', 'MAIL']
['家庭', '戶', ' ', '媽媽', '說', '女兒', '不開', '了', '，', '因為', '打工', '收入', '不', '穩定']
['推', 'Q1']
['兒子', '要', '開戶', '未成年', ',', '不願辦', '網路', '要', '單筆', '買', '亞太', '複合', '債', '.']
['林', 'O', '屏', '216022', ' ', '家庭', '戶', ' ', '有', '可能', '有', '這戶', '下單', '又', '嫌', '麻煩']
['台幣', ' ', '瑞興', ' ', '轉介', '208']
['想', '買', '基金', ' ', '推', 'EC', ' ', '轉介', '之']
['某', '小姐', '要', '買', 'nnl', '新興', '市場', '債', ' ', '美元', '買', ' ']
['[', 'ECP', ']', '可能', '買回', ' ', ' ', '積', ' ', '單', ' ', '可以', '忍', '波動']
['Idris', '轉介']
['客戶', '來電', '說', '看到', '4', '年', 'IPO', ' ', '要', '美金', '的', ' ', '先寄', '開戶', '文件']
['去電', '確認', '外幣', '帳號', '是', '台北', '富邦']
['要', '用', '女兒', '名字', '開']


From the above we can see what is segmented. It is apparently some kind of junk. SO, instead, we search for keywords.

In [22]:
key_words = dict()
cnt = 0
for i in data:
    for j in jieba.analyse.extract_tags(i[3]):
        if j in key_words:
            key_words[j] += 1
        else:
            key_words[j] = 1
    cnt = cnt + 1
    if cnt <=15:
        print(jieba.analyse.extract_tags(i[3]))

key_words = sorted(key_words.items(),key=operator.itemgetter(1))

['開戶', 'ECP', '去電', '確認', '客戶', '後來', '保單', '文件', '美元']
['ECP', '不方', 'IPO', 'MAIL']
['ECP', 'IPO', 'MAIL']
['媽媽', '女兒', '不開', '因為', '穩定', '打工', '家庭', '收入']
['Q1']
['兒子', '開戶', '不願辦', '網路', '單筆', '亞太', '複合', '未成年']
['216022', '這戶', '下單', '麻煩', '家庭', '可能']
['台幣', '瑞興', '轉介', '208']
['EC', '轉介', '基金']
['nnl', '新興', '市場', '小姐', '美元']
['ECP', '買回', '波動', '可能', '可以']
['Idris', '轉介']
['先寄', '客戶', '來電', 'IPO', '開戶', '美金', '文件', '看到']
['去電', '確認', '外幣', '帳號', '富邦', '台北']
['女兒', '名字']


Now the above seems much more readable, although we may lose some of the information in the cutting stage. Some first thoughts upon seeing these datas.
1. It is mostly key word oriented and thus many verbs are removed
2. Given that it does mention some commodities, it is still hard to decide its intention.
    * For example, '覺得', '計價', '6%', '收益', '美金', '短期', '全球' we don't know whether the customer wants to long or short 美金.
3. There are unimportant verbs concerning selling commodities.
    * 解釋, '收到', '請客'

The following blocks will show the most commonly appeared word in the service record.

In [23]:
for i in range(1,20):
    
    print(key_words[-i])

('ECP', 441)
('名單', 180)
('開戶', 163)
('Money', 148)
('客戶', 122)
('專刊', 92)
('退休', 84)
('訂閱', 78)
('電子報', 78)
('寄送', 76)
('追蹤', 74)
('分配', 65)
('IPO', 65)
('EC', 62)
('轉介', 59)
('寄回', 57)
('家庭', 54)
('收到', 47)
('表單', 46)


The above method doesn't seems to be a good idea. Since, applying key word count to different customer is useless.